## Задание 1

Немного переделал условия задачи, так как в исходном задании было непонятно как классифицировать фильмы с рейтингом от 4.0 до 4.5. Классификация фильмов предполагает рассчет средней оценки каждого фильма, но об этом явно не сказано. Если выставлять категорию по текущей оценке фильма, то получается, что мы классифицируем не фильмы, а классифицируем оценки к фильмам. А средняя оценка по фильму может попадать в диапазон > 4.0 и < 4.5 и эти фильмы по исходным данным ни в какую категорию не попадут.

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:

- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и меньше - высокий рейтинг
- оценка 5 и меньше - сверхвысокий рейтинг

Для выставления качественной оценки требуется посчитать среднюю оценку для фильма и записать его в стобце movie_avg_rating

Результат качественной классификации запишите в столбец class.

In [1]:
import pandas as pd
import re
from types import MappingProxyType

In [2]:
# Формируем датафрейм для файла movies.csv, в индекс отправляем столбцец movieId
df_movies = pd.read_csv('ml-latest-small/movies.csv', index_col='movieId')

# Формируем датафрейм для файла ratings.csv, в индекс отправляем столбцец movieId
df_ratings = pd.read_csv('ml-latest-small/ratings.csv', index_col='movieId')

# Объединяем датафреймы с фильмами и рейтингами по индексу movieId
# Так как нам нужны только те фильмы, у которых есть рейтинг, то для объединения выбираем метод inner
# Объединяем по индексу, так как в индексе находятся равные идентификаторы фильмов
df_movies_ratings = pd.merge(df_movies, df_ratings, right_index=True, left_index=True)

print('Таблица с рейтингами фильмов')
display(df_movies_ratings[:5])

Таблица с рейтингами фильмов


,title,genres,userId,rating,timestamp
movieId,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,3.0,851866703
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9,4.0,938629179
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13,5.0,1331380058
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.0,997938310
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19,3.0,855190091


In [3]:
def set_rating_category(elem, column_name:str='ratings') -> str:
    """
    Функция для определения категории рейтинга исходя из его оценки:
        - оценка 2 и меньше - низкий рейтинг
        - оценка 4 и меньше - средний рейтинг
        - оценка 4.5 и меньше - высокий рейтинг
        - оценка 5 и меньше - сверхвысокий рейтинг
        
    Предполагается, что перед использованием функции в датафрейме уже сделали группировку по фильмам 
    и усреднили оценку. Функцию можно применять к отдельному столбцу или строке датафрейма.
        
    Входные параметры:
        elem -- строка датафрейма с типом Series или значение столбца с типом int или float
        column_name -- название индекса, в котором хранятся значения рейтинга. Используется только 
                       в случае значения elem в формате Series
                       
    Возвращаемые значения:
        категория рейтинга фильма, тип str
        
    Виды категорий рейтинга:
        - low
        - middle
        - high
        - awesome
        - пустая строка в случае ошибки
    """
    
    elem_is_Series = isinstance(elem, pd.core.series.Series)
    elem_is_number = isinstance(elem, int) or isinstance(elem, float)
    
    if (not elem_is_Series and not elem_is_number):
        raise TypeError("Параметр elem должен иметь тип int, float или pandas.core.series.Series") 
    
    # Переменную column_name преобразуем в строку, так как функция ожидает здесь значение str
    column_name = str(column_name)
        
    # Создаем переменную для анализа в завивимости от типа параметра elem
    if elem_is_number:
        rating_value = elem
    else:
        if column_name not in elem.index:
            raise ValueError(f"Значение column_name='{column_name}' отсутствует в индексе объекта elem")
        rating_value = elem[column_name]
        
    if 0 <= rating_value <= 2:
        return 'low'
    if 2 < rating_value <=4:
        return 'middle'
    if 4 < rating_value <=4.5:
        return 'high'
    if 4.5 < rating_value <=5:
        return 'awesome' 
    
    # Если rating_value не входит в диапазон от 0 до 5, считаем это некорректным и возвращаем пустую строку
    return ''

In [4]:
# рассчитаем среднюю оценку для каждого фильма и запишем ее в столбец avg_rating
df_movies_ratings['movie_avg_rating'] = df_movies_ratings.groupby(by='movieId').mean()['rating']

# На основе средней оценки получим категорию рейтинга для каждого фильма
df_movies_ratings['class'] = df_movies_ratings.apply(set_rating_category, axis=1, 
                                                     column_name='movie_avg_rating')

In [5]:
# Проверяем были ли ошибки во время определении категории рейтнига
# В случае возникновения ошибок категория будет заполнена пустой строкой
class_errors_count = len(df_movies_ratings[df_movies_ratings['class'] == ''])
if class_errors_count == 0:
    print('Не найдено ошибок при создании категории рейтинга фильма в столбце class')
else:
    print(f'Кол-во ошибок формирования категории рейтинга фильма: {class_errors_count}')

Не найдено ошибок при создании категории рейтинга фильма в столбце class


In [6]:
# Выведем по 5 фильмов из каждого класса
display(df_movies_ratings[['title', 'movie_avg_rating', 'class']]
        .drop_duplicates().groupby(by='class').head(5))

,title,movie_avg_rating,class
movieId,,,
1,Toy Story (1995),3.872470,middle
2,Jumanji (1995),3.401869,middle
3,Grumpier Old Men (1995),3.161017,middle
4,Waiting to Exhale (1995),2.384615,middle
5,Father of the Bride Part II (1995),3.267857,middle
26,Othello (1995),4.100000,high
28,Persuasion (1995),4.083333,high
29,"City of Lost Children, The (Cité des enfants p...",4.025000,high
30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,4.050000,high


## Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

```
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```

Результат классификации запишите в отдельный столбец region.

In [7]:
# Так как ключи словаря являются хешируемым элементом и поиск по ключам осуществляется значительно быстрее, 
# чем по значениям словаря, то принято решение сделать в качестве ключей названия городов, 
# это повысит скорость поиска нужного города.
# Чтобы функция каждый раз не создавала словарь, принято решение сделать гео-дату в виде глобальной переменной
GEO_DATA = {
    'москва': 'Центр',
    'тула': 'Центр',
    'ярославль': 'Центр',
    'петербург': 'Северо-Запад',
    'псков': 'Северо-Запад',
    'мурманск': 'Северо-Запад',
    'владивосток': 'Дальний Восток',
    'сахалин': 'Дальний Восток',
    'хабаровск': 'Дальний Восток',  
}


def geo_class(elem, geo_data, column_name:str='keyword') -> str:
    """
    Функция для определения географической принадлежности строки с символами.
    
    Функцию можно применять к отдельному столбцу или строке датафрейма pandas.
        
    Входные параметры:
        elem -- строка датафрейма с типом Series или значение столбца с типом str
        column_name -- название индекса, в котором хранятся строковые значения для анализа. 
                       Используется только в случае значения elem в формате Series
        geo_data -- словарь с гео-датой в формате {'название города': 'гео-признак'}
                       
    Возвращаемые значения:
        географическая метка, тип str
    """
    
    elem_is_Series = isinstance(elem, pd.core.series.Series)
    elem_is_str = isinstance(elem, str)
    
    if (not elem_is_Series and not elem_is_str):
        raise TypeError("Параметр elem должен иметь тип str или pandas.core.series.Series") 
        
    if not isinstance(geo_data, dict):
        raise TypeError("Параметр geo_data должен иметь тип dict") 
        
    # Переменную нельзя менять внутри функции, поэтому делаем проксирование с правом только на чтение
    geo_data = MappingProxyType(geo_data)
    
    # Переменную column_name преобразуем в строку, так как функция ожидает здесь значение str
    column_name = str(column_name)
        
    # Создаем переменную для анализа в завивимости от типа параметра elem
    if elem_is_str:
        str_value = elem
    else:
        if column_name not in elem.index:
            raise ValueError(f"Значение column_name='{column_name}' отсутствует в индексе объекта elem")
        str_value = elem[column_name]
  
    # Разбиваем строку на отдельные слова
    word_list = re.split(r'[\.\,\; ]', str_value)
    
    city_list = geo_data.keys()
    
    # Ищем слова в списке городов, если находим город в ключах словаря, то выдаем значение с гео-меткой
    for word in word_list:
        word = word.lower().strip()
        if word in city_list:
            return geo_data[word]
        
    # Если ни одно слово не совпало с городом из словаря или word_list пустой, то возвращаем undefined
    return 'undefined'

In [8]:
# Формируем датафрейм с ключевыми словами
df_keywords = pd.read_csv('keywords.csv')

In [9]:
# Раставляем гео-метки для ключевых слов в колонку region
df_keywords['region'] = df_keywords.apply(geo_class, axis=1, column_name='keyword', geo_data=GEO_DATA)

In [10]:
# Выведем по 5 ключевых слов из каждой категории
display(df_keywords.groupby(by='region').head(5))

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад


## Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

    2.1 для каждой строки пройдите по всем годам списка years
    
    2.2 если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    
    2.3 если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
    
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

Немного переделал алгоритм:

1. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Предполагаем, что год выпуска указан в формате (YYYY) и может находиться в любой части названия фильма. Если найдено больше одного вхождения формата года или не найдено ничего, то функция должна вернуть год 1700.
2. Запишите год выпуска фильма по алгоритму пункта 1 в новый столбец ‘year’
3. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [11]:
def get_movie_year(elem, column_name=None):
    """
    Функция определяет год выпуска фильма с помощью анализа названия фильма.
    
    Функцию можно применять к отдельному столбцу или строке датафрейма pandas.
    
    Входные параметры:
        elem -- строка датафрейма с типом Series или значение столбца с типом str
        column_name -- название индекса, в котором хранятся значения рейтинга. Используется только 
                       в случае значения elem в формате Series
    """
    
    elem_is_str = isinstance(elem, str)
    elem_is_Series = isinstance(elem, pd.core.series.Series)
    
    if (not elem_is_Series and not elem_is_str):
        raise TypeError("Параметр elem должен иметь тип str или pandas.core.series.Series") 
    
    # Переменную column_name преобразуем в строку, так как функция ожидает здесь значение str
    column_name = str(column_name)
    
    # Создаем переменную для анализа в завивимости от типа параметра elem
    if elem_is_str:
        movie_name = elem
    elif elem_is_Series:
        if column_name not in elem.index:
            raise ValueError(f"Значение column_name='{column_name}' отсутствует в индексе объекта elem")
        movie_name = elem[column_name]
    
    #Опытным путем выявлено, что меньше всего ошибок возникает, если искать год в формате (YYYY)
    movie_years_list = re.compile(r'\((\d{4})\)').findall(movie_name)
    
    # Если найдено более двух годов в имени фильма или не найдено ничего, то возвращаем 1990 год.
    if len(movie_years_list) > 1 or len(movie_years_list) == 0:
        return 1700
    
    return int(movie_years_list[0])

In [12]:
# Для каждого фильма найдем год фильма и запишем его в столбец year
df_movies_ratings['year'] = df_movies_ratings.title.apply(get_movie_year)

In [13]:
# Формируем отдельный датафрейм с индексом year и столбцом rating со средним рейтингом всех фильмов по году
df_year_movie_ratings = df_movies_ratings[['year', 'rating']].groupby('year').mean()

In [14]:
# Топ-20 годов из диапазона от 1950 по текущий, в которые были сняты фильмы с наилучшими рейтингами
display(df_year_movie_ratings[df_year_movie_ratings.index > 1950].sort_values('rating', ascending=False)[:20])

,rating
year,
1957,4.014241
1972,4.011136
1952,4.000000
1954,3.994220
1951,3.983539
1974,3.978704
1962,3.952446
1977,3.905786
1964,3.841492
